# PyTorch index methods

파이토치에서 인덱스를 가지고 놀다 보면 헷갈릴 때가 많다. 하나씩 정리해보자.
한번에 다 정리하긴 힘들 거 같고 그때그때 하나씩 한다.

Candidates:

- `gather`
- `scatter`

또 있나?

### Additional library

- https://github.com/rusty1s/pytorch_scatter

<p align="center">
  <img width="30%" src="https://raw.githubusercontent.com/rusty1s/pytorch_scatter/master/docs/source/_figures/add.svg?sanitize=true" />
</p>

뭐 요런걸 해준다고 함!

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

## Basic

In [9]:
idx = torch.as_tensor([0, 2])

- 특정 차원을 전부 지정하고 싶다면 `:`

In [10]:
t = torch.zeros(3, 3, 3)
t[:, idx] = 1.
t

tensor([[[1., 1., 1.],
         [0., 0., 0.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [1., 1., 1.]]])

- 각 차원마다 인덱스를 지정해주고 싶다면
  - `t[i, j] = 1.` where `i` in `idx0` and `j` in `idx` 를 하고 싶다면

In [12]:
t = torch.zeros(3, 3, 3)
idx0 = torch.as_tensor([1, 2])
t[idx0, idx] = 1.
t

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [1., 1., 1.]]])

- one-hot vector 같은건 요렇게 만들 수:

In [15]:
t = torch.zeros(4, 4)
labels = torch.as_tensor([0, 3, 1, 1])

t[torch.arange(4), labels] = 1.
t

tensor([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

## Scatter

scatter 는 일단 공식 도큐먼트에는 다음과 같이 나온다:

Tensor.scatter_(dim, index, src, reduce=None) → Tensor

```python
# 3d 일때 예제.
self[index[i][j][k]][j][k] = src[i][j][k]  # if dim == 0
self[i][index[i][j][k]][k] = src[i][j][k]  # if dim == 1
self[i][j][index[i][j][k]] = src[i][j][k]  # if dim == 2
```

- 대충 이런거다. 즉, src -> self 로 카피를 해 오고 싶은데, 이때 그대로 카피해오고 싶은게 아니라 index 에 따라 위치를 바꾸고 싶은 것.
- 다만 위 예제로 다 설명되는 건 아님. 일단 `src` 대신 `value` 로 특정 스칼라값 지정도 가능함. 혹은 keyword 를 안쓰면 알아서 작동한다.
- 그리고 위처럼 보면 무조건 `self` 와 `src` 의 shape 이 같아야 할 것 같지만 그렇지도 않다.
  - 일단 차원 수는 다 같아야 함
  - `index.size(d) <= src.size(d)` for all `d`
  - `index.size(d) <= self.size(d)` for all `d != dim`
    - `d == dim` 일때는, 아마도, 더 커도 상관이 없는 듯. 그만큼 더 assign 을 할 뿐이지 값이 range 안에만 있으면 문제될 거 없음. 근데 더 클 일이 있나...?
  - `index` 와 `src` 는 broadcast 되지 않는다.

- <span style="color:red"> **!! Caution !!** </span> : The backward pass is implemented only for `src.shape` == `index.shape`.


- 기본 예제

In [39]:
"""
dim = 0 이므로, 기본적으로
self[index[i, j], j] = src[i, j] 다.

즉, j 를 따라 self 의 i 축에 src 를 박아넣자는 게 기본 골자이고,
이 때 i 를 그냥 i 가 결정하는 게 아니라 index 가 결정하게 하겠다는 것.

그래서 output shape 은 당연히 self 지만,
이 output 에 "얼마나 박아넣냐" 는 index 만큼임.
"""

src = torch.arange(1, 11).reshape(2, 5)
index = torch.as_tensor([[0, 1, 2, 0]])
slf = torch.zeros(3, 5, dtype=src.dtype)
print("self/index/src:", slf.shape, index.shape, src.shape)
print(src)
slf.scatter_(0, index, src)

self/index/src: torch.Size([3, 5]) torch.Size([1, 4]) torch.Size([2, 5])
tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])


tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])

In [40]:
"""
여기서는 dim=1 을 줬다.
따라서 self[i, index[i, j]] = src[i, j] 가 되는 것.
"""

index = torch.as_tensor([[0, 1, 2], [0, 1, 4]])
slf = torch.zeros(3, 5, dtype=src.dtype)
slf.scatter_(1, index, src)

tensor([[1, 2, 3, 0, 0],
        [6, 7, 0, 0, 8],
        [0, 0, 0, 0, 0]])

src 까지 생각하면 좀 더 헷갈리는 것 같고... src 를 빼고 생각하면 좀 더 단순하다.

- 이번엔 응용 예제를 해 보자. 2d index 를 3d one-hot 으로 확장하는 것.
  - 위에서 2d one-hot 은 torch.arange 로 간단하게 만들 수 있었지만 3d 로 가면 그게 어려움.

In [50]:
index = torch.randint(0, 3, [1, 3, 3])  # 2d index
print(index.squeeze(-1))
slf = torch.zeros(3, 3, 3, dtype=src.dtype)
slf.scatter_(0, index, 1)  # 3d one-hot

tensor([[[2, 2, 2],
         [1, 0, 2],
         [2, 1, 2]]])


tensor([[[0, 0, 0],
         [0, 1, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 1, 0]],

        [[1, 1, 1],
         [0, 0, 1],
         [1, 0, 1]]])

### Scatter w/ reduce

단순히 assign 하는것 외에도 `add` 와 `multiply` 의 reduce 연산이 가능하다. 특히 `add` 는 `scatter_add_` 라고 따로 함수까지 있음.

예제 상황: example-wise loss 들이 있을 때, 이걸 class-wise mean 을 하고 싶다면?

In [105]:
n_classes = 3
B = 4

t = torch.rand(B, 7)
# idx = torch.as_tensor([2, 0, 0, 0])
idx = torch.randint(n_classes, [B])
s = torch.zeros([n_classes])
t.mean(1)

tensor([0.5146, 0.5474, 0.6356, 0.6951])

In [106]:
# want to do: s[idx] += t.mean(1)
print(s)
print(idx)
print(t.mean(1))

tensor([0., 0., 0.])
tensor([2, 2, 2, 1])
tensor([0.5146, 0.5474, 0.6356, 0.6951])


In [107]:
# for문으로 하면 이렇게
s = torch.zeros([n_classes])
for i, v in zip(idx, t.mean(1)):
    s[i.item()] += v
print(s)

tensor([0.0000, 0.6951, 1.6976])


In [108]:
s = torch.zeros([n_classes])
s.scatter_(0, idx, t.mean(1), reduce='add')

tensor([0.0000, 0.6951, 1.6976])

In [109]:
s = torch.zeros([n_classes])
s.scatter_add_(0, idx, t.mean(1))

tensor([0.0000, 0.6951, 1.6976])

In [110]:
# 여기까지는 class-wise sum 을 한 거고 mean 을 하려면 count 만큼 나눠줘야 한다.
s.div(idx.bincount(minlength=n_classes))

tensor([   nan, 0.6951, 0.5659])

In [111]:
r = s.div(idx.bincount(minlength=n_classes))
r[r.isfinite()].mean()

tensor(0.6305)